## Observations and Insights 

In [111]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

mouse_metadata.head()

# Combine the data into a single dataset
total_study_df = pd.merge(mouse_metadata, study_results, how='left', on='Mouse ID')

total_study_df.head()
# Display the data table for preview


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [62]:
total_study_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1893 entries, 0 to 1892
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Mouse ID            1893 non-null   object 
 1   Drug Regimen        1893 non-null   object 
 2   Sex                 1893 non-null   object 
 3   Age_months          1893 non-null   int64  
 4   Weight (g)          1893 non-null   int64  
 5   Timepoint           1893 non-null   int64  
 6   Tumor Volume (mm3)  1893 non-null   float64
 7   Metastatic Sites    1893 non-null   int64  
 8   index-col           1893 non-null   int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 147.9+ KB


In [31]:
len(mouse_metadata)

249

In [34]:
len(study_results)

1893

In [82]:
merging = mouse_metadata.groupby('Drug Regimen').count()
merging["Mouse ID"].sum()

249

In [39]:
pd.set_option('display.max_rows', total_study_df.shape[0]+1)

In [21]:
total_study_df.columns

Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)',
       'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites'],
      dtype='object')

In [69]:
check_for_error = total_study_df['Mouse ID']

In [29]:
check_for_error.sort_values()
check_for_error.value_counts()
len(check_for_error.value_counts())
#Number of Mice

249

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 



In [4]:
# Optional: Get all the data for the duplicate mouse ID. 



In [83]:
total_study_df.columns

Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)',
       'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites', 'index-col'],
      dtype='object')

In [127]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
total_study_df = total_study_df[['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)', 'Tumor Volume (mm3)', 'Metastatic Sites', 'Timepoint']]

total_study_drop_df = total_study_df.drop_duplicates(subset=['Mouse ID'], keep='last')

total_study_drop_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3),Metastatic Sites,Timepoint
9,k403,Ramicane,Male,21,16,22.050126,1,45
19,s185,Capomulin,Female,3,17,23.343598,1,45
29,x401,Capomulin,Female,16,15,28.484033,0,45
39,m601,Capomulin,Male,22,17,28.430964,1,45
49,g791,Ramicane,Male,11,16,29.128472,1,45


In [103]:
# Checking the number of mice in the clean DataFrame.
len(total_study_df["Mouse ID"])


249

## Summary Statistics

In [128]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
study_mean = total_study_drop_df['Tumor Volume (mm3)'].mean() 
study_median = total_study_drop_df['Tumor Volume (mm3)'].median()
# This method is the most straighforward, creating multiple series and putting them all together at the end.
study_variance = np.var(total_study_drop_df['Tumor Volume (mm3)'])
study_std = total_study_drop_df['Tumor Volume (mm3)'].std()
study_sem = total_study_drop_df['Tumor Volume (mm3)'].sem()

study_dict = {"Mean":[study_mean], "Median":[study_median], "Variance":[study_variance], 
             "Standard Deviation": [study_std], "SEM": [study_sem]}

study_dict_df = pd.DataFrame(study_dict)

study_dict_df

,Mean,Median,Variance,Standard Deviation,SEM
0,54.999137,55.84141,159.507877,12.65508,0.801983


In [155]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

mean_df = total_study_drop_df.groupby(['Drug Regimen']).mean()
mean_df = mean_df['Tumor Volume (mm3)']
median_df = total_study_drop_df.groupby(['Drug Regimen']).median()
median_df = median_df['Tumor Volume (mm3)']

variance_df = total_study_drop_df.groupby(['Drug Regimen']).var()
variance_df = variance_df['Tumor Volume (mm3)']
variance_df

std_df = total_study_drop_df.groupby(['Drug Regimen']).std()
std_df = std_df['Tumor Volume (mm3)']
std_df

sem_df = total_study_drop_df.groupby(['Drug Regimen']).sem()
sem_df = sem_df['Tumor Volume (mm3)']
sem_df


# This method produces everything in a single groupby function
summary_df = pd.merge(mean_df, median_df, on="Drug Regimen")
summary_df = pd.merge(summary_df, variance_df, on="Drug Regimen")
summary_df = summary_df.rename(columns={"Tumor Volume (mm3)_x":"Mean",
                                        "Tumor Volume (mm3)_y":"Median",
                                        "Tumor Volume (mm3)":"Variance"})
summary_df = pd.merge(summary_df, std_df, on="Drug Regimen")
summary_df = summary_df.rename(columns={"Tumor Volume (mm3)":"Std"})
summary_df = pd.merge(summary_df, sem_df, on="Drug Regimen")
summary_df = summary_df.rename(columns={"Tumor Volume (mm3)":"SEM"})


summary_df
#summary_df

,Mean,Median,Variance,Std,SEM
Drug Regimen,,,,,
Capomulin,36.667568,38.125164,32.663378,5.715188,1.143038
Ceftamin,57.753977,59.851956,69.982735,8.365568,1.673114
Infubinol,58.178246,60.165180,74.010875,8.602957,1.720591
Ketapril,62.806191,64.487812,98.921330,9.945920,1.989184
Naftisol,61.205757,63.283288,106.029927,10.297083,2.059417
Placebo,60.508414,62.030594,78.759797,8.874672,1.774934
Propriva,56.736964,55.841410,69.349002,8.327605,1.665521
Ramicane,36.191390,36.561652,32.166354,5.671539,1.134308
Stelasyn,61.001707,62.192350,90.331586,9.504293,1.940056


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
